# Sistema de Scrap para Clasificados de Diarios

### Objetivo:
Preparar y organizar la información obtenida de los clasificados de diario Clarin desde 19.. a 19..

### Herramientas Utilizadas:
- Python
- Tesseract-OCR (pytesseract)
- SQLite3 (almacenamiento de la información)
- Pandas (organizacion de la información)

## Desarrollo

### Importación de las librerias y paquetes a utilizar

In [22]:
from PIL import Image
import pandas as pd
import pytesseract
import cv2
from src.utils.databaseConnection import crearConexion
from src.utils import sqlQueries

### Preparación de la imagen

In [23]:
img = cv2.imread("data/import/recorteEjemplo.png")
#Resize the image
print(img.shape)

scale_percent = 300 # percent of original size
width = int(img.shape[1] * scale_percent / 100)
height = int(img.shape[0] * scale_percent / 100)
dim = (width, height)

img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
#Convert image to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (7,7), 0)


#Convert image to black and white
new_image = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 21, 3)
#new_image = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 85, 11)
print(new_image.shape)

cv2.imwrite('data/export/salidaImport.png', new_image) 

(471, 341, 3)
(1413, 1023)


True

### Scrap de la imagen utilizando Tesseract.

In [24]:
scrapCrudo = pytesseract.image_to_string(new_image, lang='eng+spa', config='--psm 3 load_system_dawg=false')
print('Fuente: \n')
print(scrapCrudo)

Fuente: 

P/JUNTA Colpovn 79 p 1° dio 7 2de
p/escal 3 umb c/depond y lo a/
prof 10-13 hs ir 921-5003 ©

“RAMOS: MEJIA 3 amb-tel ap pro
Rivadavia 14340 p 12° B

- R MEJIA Ja Rivedavia 13242 8”

SAN MARTIN 3 amb 55 miil Sar
. miento 1821 ver 10-12 15-17 hs

5 FE:Y Aguoro 1650 4° 27 a prof
amb TE bñoa comp kilch a/acon(
helad moa lum v 14-17 SoriCo

V.CRESPO Lovalleja 64 8° B 2
- AaMmbcoates lav v 14-18 50-2598

VILLA CRESPO nuavo 5 amb ca
Ue SII mill. Levitt 855-6645

yv OLE 2a calla PB 25th Ma avo

- A24 Din A-54.500.000 2 eortos. V.
?2-12.15-18 F.Prop 797-6246

Vv LOPEZ H Yrigoyen y Malrú don
t/caza 3 amb 1%p:ftc balciza te- )

_ léf lav cto pogo ad 824-2783

V.URQUIZA 3 amb ntio txo PORO, ”
total 18 meses 51:6509/0322

WARNES 73 p 6° 8 nuevo: iIndos 3
amb 30.500.000 ver 15-17 hz A



## Transformación de los datos para la base

Estructura de la informacion en la base de datos.

In [25]:
data = {'barrio': [],
        'direccion': [],
        'precio': [],
        'tipo': [],
        'caracteristicas': [],
        'otros': []
        }

In [27]:
separadoPorEspacios = scrapCrudo.split('\n\n')

primerAviso = separadoPorEspacios[0]
print(primerAviso)
primerAvisoPorEspacios = primerAviso.replace('\n', ' ')
primerAvisoPorEspacios = primerAvisoPorEspacios.split(' ')

for i in range(len(primerAvisoPorEspacios)-1):
    for charac in primerAvisoPorEspacios[i]:
        if charac.isdigit():
            primerAvisoPorEspacios[i] = primerAvisoPorEspacios[i] + ' ' + primerAvisoPorEspacios[i+1]
            primerAvisoPorEspacios[i+1] = ''
            break

primerAvisoPorEspacios = [i for i in primerAvisoPorEspacios if i != '']

for key in data.keys():
    data[key].append(primerAvisoPorEspacios[0])
    primerAvisoPorEspacios.pop(0)


df = pd.DataFrame(data, columns= ['barrio', 'direccion', 'precio', 'tipo', 'caracteristicas', 'otros'])
print(primerAvisoPorEspacios)
print(df)

P/JUNTA Colpovn 79 p 1° dio 7 2de
p/escal 3 umb c/depond y lo a/
prof 10-13 hs ir 921-5003 ©
['3 umb', 'c/depond', 'y', 'lo', 'a/', 'prof', '10-13 hs', 'ir', '921-5003 ©']
    barrio direccion precio    tipo caracteristicas    otros
0  P/JUNTA   Colpovn   79 p  1° dio           7 2de  p/escal
1  P/JUNTA   Colpovn   79 p  1° dio           7 2de  p/escal


### Carga de la informacion en la base de datos

In [10]:
cursor = crearConexion()

df.to_sql('alquileres', cursor, if_exists='append', index = False)
 
alquileresDF = sqlQueries.obtenerDfFromQuery(sqlQueries.obtenerAlquileres(), cursor)

print(alquileresDF)

    barrio direccion precio    tipo caracteristicas    otros
0  P/JUNTA   Colpovn   79 p  1° dio           7 2de  p/escal
